# Eliminacion Recursiva de Features (RFE)

Es una tecnica utilizada para seleccionar las caracteristicas mas importantes de un conjunto de datos. Su objetivo es eliminar las caracteristicas menos relevantes, mejorando asi el rendimiento del modelo y reduciendo su complejidad.

Generalmente, RFE se aplica a modelos que pueden proporcionar alguna medida de la importancia de las caracteristicas como los modelos lineales o arboles de decision. Entre los modelos donde se puede aplicar tenemos:

- Regresion Lineal
- Regresion Logistica
- Arboles de decision
- Bosques aleatorios
- Maquina de vectores de soporte
- Entre otros...

## Como funciona RFE?

   1. **Entrenamiento inicial**: Entrena un modelo utilizando todas las caracteristicas (Variables predictoras)
    
    
   2. **Evaluacion de Importancia**: Evalua la importancia de cada caracteristica. Esta importancia se puede medir de diferentes maneras dependiendo del modelo utilizado (coeficientes de la regresion lineal)
   
    
   3. **Eliminacion de Caracteristicas**: Elimina las caracteristicas menos importantes para el modelo
   
   
   4. **Repeticion de pasos**: Repite los pasos de 1 a 3 hasta alcanzar un numero predefinido de caracteristicas.

## Aplicacion de RFE

Vamos a crear un conjunto de datos de prueba de 6 variables, incluyendo la variable objetivo y vamos a usar RFE para seleccionar las 3 mas importantes para el modelo.

In [133]:
# Generamos datos de prueba
import numpy as np
import pandas as pd

# semilla
np.random.seed(0)

# 5 caracteristicas
X = pd.DataFrame({
    'feature_1': np.random.rand(100),
    'feature_2': np.random.rand(100),
    'feature_3': np.random.rand(100),
    'feature_4': np.random.rand(100),
    'feature_5': np.random.rand(100)
})


Para probar el metodo RFE, asociamos linealmente la variable objetivo con 3 de las 5 caracteristicas: feature 1, 2 y 5. Esto nos asegura que el modelo solo va a estar influenciado por estas tres caracteristicas lo cual debe verse reflejado al aplicar RFE.

In [134]:
# variable objetivo
y = 3*X['feature_2'] + 2*X['feature_5'] + X['feature_1'] + np.random.rand(100)

Ahora, juntamos los datos en un Dataframe para mejorar la visualizacion de los datos

In [135]:
df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(X.shape[1])])
df['target'] = y
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,target
0,0.548814,0.677817,0.311796,0.906555,0.401260,3.695163
1,0.715189,0.270008,0.696343,0.774047,0.929291,3.756831
2,0.602763,0.735194,0.377752,0.333145,0.099615,3.532546
3,0.544883,0.962189,0.179604,0.081101,0.945302,6.072647
4,0.423655,0.248753,0.024679,0.407241,0.869489,3.242399


### 1.- Entrenamiento inicial
Entrenamos el modelo de regresion lineal con todas las caracteristicas y calculamos la importancia de cada una.

In [136]:
# libreria para usar la regresion lineal
from sklearn.linear_model import LinearRegression

# funcion para entrena el modelo
def train_model(X, y):
    model = LinearRegression()
    return model.fit(X,y)

In [137]:
# entrenamos el modelo
modelo = train_model(X, y)

### 2.- Evaluacion de importancia
Para el caso de la regresion, debemos obtener los coeficientes de cada caracteristicas una vez entrenado el modelo.

In [138]:
# funcion para obtener los coeficientes de cada caracteristica
def evaluate_model(model):
    coeficientes = model.coef_
    return coeficientes

In [139]:
# coeficiente de las caracteristicas
coefi = evaluate_model(modelo)
coefi

array([1.0169788 , 2.96313024, 0.01281724, 0.04742558, 1.94336239])

### 3.- Eliminacion de caracteristicas
Identificamos la caracteristica con el coeficiente mas pequeño (el valor absoluto) y la eliminamos

In [140]:
# funcion que determina la caracteristica con menor valor de coeficiente y la elimina
def remove_feature(X, coeficientes):
    # indice del coficiente mas pequeño
    min_indice = np.argmin(np.abs(coeficientes))
    # nombre de la caracteristica correspondiente al indice anterior
    feature_remove = X.columns[min_indice]
    # eliminar caracteristica
    X = X.drop(columns=[feature_remove])
    # retornar matriz de caracteristica y nombre de la caracteristica eliminada
    return X, feature_remove
    

### 4.- Repetir hasta obtener el numero deseado de caracteristicas

Inicialmente dijimos que vamos a obtener las 3 caracteristicas mas importantes. Entonces, repetimos el proceso del 1 al 3 hasta obtener las tres caracteristicas. Para automatizar el proceso, procedemos como sigue

In [141]:
# numero de features a seleccionar
n_features = 3

# ejecutar proceso hasta llegar a 3 caracteristicas
while X.shape[1] > n_features:
    modelo = train_model(X,y)
    coefi = evaluate_model(modelo)
    X, feature_remove = remove_feature(X, coefi)
    
# mostrar las 3 caracteristicas mas importantes
print('Caracteristicas seleccionadas', X.columns)
    
    

Caracteristicas seleccionadas Index(['feature_1', 'feature_2', 'feature_5'], dtype='object')


Los resultados confirman que los features 1, 2 y 5 son los que realmente influyen sobre el modelo de regresion lineal.

## Aplicacion de RFE con Scikit-Learn

Ahora vamos a utilizar la libreria Scikit Learn para aplicar este metodo y simplificar el proceso de eliminacion recursiva. Trabajaremos sobre los mismos datos de prueba del ejemplo anterior para comparar los resultados 

In [142]:
# features
X = df.drop(columns=['target'])
# variable objetivo
y = df['target']

Aplicamos RFE con la libreria sklearn

In [143]:
# libreria para aplicar RFE
from sklearn.feature_selection import RFE

# crear modelo de regresion
modelo_2 = LinearRegression()

# Crear el objeto RFE y especificar el numero de caracteristicas deseadas
rfe = RFE(estimator=modelo_2, n_features_to_select=3)

# ajustar RFE a los datos
rfe.fit(X, y)

# obtener las caracteristicas seleccionadas por RFE
features_select = X.columns[rfe.support_]

# mostrar nombre
features_select

Index(['feature_1', 'feature_2', 'feature_5'], dtype='object')

De esta forma, obtenemos los features mas importantes para el modelo de regresion, que en este caso son los features 1, 2 y 5, coincidiendo con los resultados del ejemplo anterior.

# Eliminacion Recursiva de Features con Validacion Cruzada (RFECV)

Es una tecnica de seleccion de caracteristicas que mejora la metodologia RFE al incoporar la validacion cruzada para seleccionar automaticamente el numero optimo de caracteristicas

## Como funciona RFECV?

1. **Entrenamiento del modelo**: Similar a RFE, entrena el modelo usando todas las caracteristicas


2. **Validacion Cruzada**: Utiliza la validacion cruzada para evaluar el rendimiento del modelo en diferentes subconjuntos de datos.


3. **Eliminacion de caracteristicas**: Elimina la caracteristica menos importante y evalua el modelo usando validacion cruzada.


4. **Repeticion de pasos**: Repite los pasos 1 hasta 3 hasta que se encuentre el conjunto optimo de caracteristicas que maximiza el rendimiento del modelo.

## Aplicacion de RFECV con Scikit Learn

Vamos a utilizar un conjunto de datos de prueba de 11 caracteristicas incluyendo la variable objetivo.  

In [144]:
# semilla
np.random.seed(0)

# 10 caracteristicas
X = pd.DataFrame({
    'feature_1': np.random.rand(100),
    'feature_2': np.random.rand(100),
    'feature_3': np.random.rand(100),
    'feature_4': np.random.rand(100),
    'feature_5': np.random.rand(100),
    'feature_6': np.random.rand(100),
    'feature_7': np.random.rand(100),
    'feature_8': np.random.rand(100),
    'feature_9': np.random.rand(100),
    'feature_10': np.random.rand(100)
})

Relacionamos la variable objetivo con 4 caracteristicas: feature 1, 2, 5 y 7. Esto asegura que de las 10 caracteristicas, solo 4 influencian el modelo y por tanto, al aplicar RFECV, nos debe arrojar dichas caracteristicas.

In [145]:
# variable objetivo
y = (
    3*X['feature_2'] + 
    2*X['feature_5'] + 
    X['feature_1'] + 
    X['feature_7']*X['feature_7'] +
    np.random.rand(100)
)
y.head()

0    4.008168
1    3.501436
2    3.946276
3    6.034817
4    3.321619
dtype: float64

Juntamos los datos en un Dataframe para visualizarlos mejor

In [146]:
df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(X.shape[1])])
df['target'] = y
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,target
0,0.548814,0.677817,0.311796,0.906555,0.401260,0.310381,0.174658,0.373216,0.039993,0.810839,4.008168
1,0.715189,0.270008,0.696343,0.774047,0.929291,0.373035,0.327988,0.222864,0.639705,0.348192,3.501436
2,0.602763,0.735194,0.377752,0.333145,0.099615,0.524970,0.680349,0.080532,0.408303,0.211455,3.946276
3,0.544883,0.962189,0.179604,0.081101,0.945302,0.750595,0.063208,0.085311,0.377407,0.059383,6.034817
4,0.423655,0.248753,0.024679,0.407241,0.869489,0.333507,0.607249,0.221396,0.809365,0.876027,3.321619


Dividimos los datos en entrenamiento y prueba

In [147]:
# libreria para dividir los datos
from sklearn.model_selection import train_test_split

# entrenamiento 70% y prueba 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Aplicamos RFECV sobre los datos para un modelo de regresion lineal

In [148]:
# importamos el metodo RFECV
from sklearn.feature_selection import RFECV

# creamos el modelo de regresion lineal
model_2 = LinearRegression()

# configuramos RFECV
rfecv = RFECV(
    estimator=model_2,
    step=1, # paso para eliminar caracteristicas
    cv=5, # numero de pliegues para la validacion cruzada
    scoring='neg_mean_squared_error' # uso del MSE negativo como metrica de rendimiento
)

# ajustar el modelo utilizando RFECV
rfecv.fit(X_train, y_train)

# mostrar numero optimo de caracteristicas
print('Numero optimo de caracteristicas: ', rfecv.n_features_)
# mostrar nombre de las caracteristicas
print('Caracteristicas seleccionadas: ', X_train.columns[rfecv.support_])

Numero optimo de caracteristicas:  4
Caracteristicas seleccionadas:  Index(['feature_1', 'feature_2', 'feature_5', 'feature_7'], dtype='object')


El RFECV arroja 4 variables optimas para el modelo de Regresion Lineal: feature 1, 2, 5, y 7, mismas que han sido relacionadas al inicio con la variable objetivo y las esperadas como resultado.

RFECV recibe como parametro el *scoring* que en este caso se asigna como el negativo del error cuadratico medio (MSE). Veamos una explicacion del porque:
> *Esto se debe a como funciona la API y la convencion de algunas metricas de evaluacion. El MSE es una metrica de error, lo que significa que debemos minimizar este valor para mejorar el rendimiento del modelo. Sim embargo, la mayoria de las funciones de scikit learn estan diseñadas para maximizar la metrica de evaluacion. Por tanto al usar MSE, scikit learn lo convierte en su valor negativo para que la funcion de maximizacion funcione correctamente. Al maximizar el negativo del MSE estamos minimizando el MSE real.*